In [ ]:
# install virgin environment
!odsc conda install --uri <oci://my-bucket@my-namespace/path_to_tar_file>

# modify environment
!pip install ...

# publish in project-specific bucket final env with project-specific name
!odsc conda init -b <proj-specific-bucket-name> -n <your-tenancy-namespace> -a resource_principal
!odsc conda publish -s <slug> --force

Here we have to define the infrastructure operations for the first deploy and maybe the update of the infrastructure if a re-deploy is needed

* creation of log group and custom log to be used in data science pipeline
* creation of data science pipeline --> here we should in future define a wrapper to let data scientist define if to use 1 or more steps
* creation of project - task - application - publishing task to application - scheduler - scheduled task

in future maybe also add the monitoring of infrastructure

In [2]:
import oci
from oci.logging.models import CreateLogDetails, UpdateLogDetails, CreateLogGroupDetails
import requests
import json

# Initialize the signer with Resource Principal
resource_principal_signer = oci.auth.signers.get_resource_principals_signer()

# LOAD CONFIG

In [4]:
with open("infra_config.json", "r") as config_file:
    config = json.load(config_file)

{'compartment_id': 'a', 'b': 'b'}


# CLIENT CONFIGURATION

In [ ]:
# Create a logging client with the signer
log_client = oci.logging.LoggingManagementClient({}, signer=resource_principal_signer)

# Initialize the OCI Data Integration client with the signer
data_integration_client = oci.data_integration.DataIntegrationClient({}, signer=resource_principal_signer)

# LOGGING STEP

In [ ]:
# Step 1: Create a log group inside the compartment
log_group = log_client.create_log_group(
    CreateLogGroupDetails(
        compartment_id=config['compartment_id'],
        display_name=config['log_group_display_name'],
        defined_tags=config['defined_tags']
    )
)

# Step 2: Retrieve OCID for the created log group
list_log_groups_response = log_client.list_log_groups( 
    compartment_id=config['compartment_id'],
    sort_by="displayName",
    sort_order="DESC")

for log_group in list_log_groups_response.data:    
    if log_group.display_name == config['log_group_display_name']:
        log_group_id = log_group.id
        
# Step 3: Create a custom log inside the created log group
log = log_client.create_log(
    log_group_id=log_group_id,
    create_log_details=CreateLogDetails(
        display_name=config['custom_log_display_name'],
        log_type="CUSTOM",
        retention_duration=30,
        defined_tags=config['defined_tags']
    )
)

# Step 4: Retrieve custom log OCID
list_log_groups_response = log_client.list_logs( 
    log_group_id=log_group_id,
    sort_by="displayName",
    sort_order="DESC")

for log in list_log_groups_response.data:    
    if log.display_name == config['custom_log_display_name']:
        log_id = log.id

### PIPELINE DEFINITION

### DATA INTEGRATION 

In [ ]:
# Step 1: Create a Data Integration project
project = data_integration_client.create_project(
    workspace_id=config['workspace_id'],
    create_project_details=oci.data_integration.models.CreateProjectDetails(
        name=config['odi_project_name']
    )
)

